# Handling SC2017 PRP productions

This module has functions to:
- create all the workload
- move from state to state [created,training,trained,generating,generated,transporting,done]


In [13]:
from time import time
from elasticsearch import Elasticsearch, exceptions as es_exceptions
es = Elasticsearch(['atlas-kibana.mwt2.org:9200'],timeout=60)
index_name='sc2017'

 [--nb-epochs NB_EPOCHS] 
 [--batch-size BATCH_SIZE]
 [--latent-size LATENT_SIZE] 
 [--disc-lr DISC_LR]
 [--gen-lr GEN_LR] 
 [--adam-beta ADAM_BETA] 
 [--prog-bar]
 [--no-attn] 
 [--debug] 
 [--d-pfx D_PFX] 
 [--g-pfx G_PFX]
 dataset

In [14]:
def create_workload():
    
    # clean all
    try:
        es.indices.delete(index='sc2017')
    except:
        print("not there?")
    
    id=0
    
    l0 = ['--nb-epochs=50','--nb-epochs=100','--nb-epochs=200',]
    l1 = ['--disc-lr=0.0001','--disc-lr=0.0002','--disc-lr=0.0005']
    l2  = ['--gen-lr=0.00001','--gen-lr=0.00002','--gen-lr=0.00005']
    l3    = ['', '--no-attn']
    l4    = ['gamma.yaml', 'eplus.yaml', 'pion.yaml']
    for a in l0:
        for b in l1:
            for c in l2:
                for d in l3:
                    for e in l4:
                        doc={}
                        doc['created']=time()
                        doc['status']='created'
                        doc['training_options']=['--output_folder=/data/CaloGAN/weigths/'+str(id), a, b, c, d, e]
                        doc['generating_options']=[
                                                   '--input_folder=/data/CaloGAN/weigths/'+str(id), 
                                                   '--output_folder=/data/CaloGAN/outputs/'+str(id), 
                                                   '--sets=10', 
                                                   '--showers=100000'
                                                  ]
                        id+=1
                        print(doc)
                        es.create(index=index_name, doc_type='doc', id=id, body=doc)
create_workload()

{'created': 1509816944.8834789, 'generating_options': ['--input_folder=/data/CaloGAN/weigths/0', '--output_folder=/data/CaloGAN/outputs/0', '--sets=10', '--showers=100000'], 'training_options': ['--output_folder=/data/CaloGAN/weigths/0', '--nb-epochs=50', '--disc-lr=0.0001', '--gen-lr=0.00001', '', 'gamma.yaml'], 'status': 'created'}
{'created': 1509816948.965161, 'generating_options': ['--input_folder=/data/CaloGAN/weigths/1', '--output_folder=/data/CaloGAN/outputs/1', '--sets=10', '--showers=100000'], 'training_options': ['--output_folder=/data/CaloGAN/weigths/1', '--nb-epochs=50', '--disc-lr=0.0001', '--gen-lr=0.00001', '', 'eplus.yaml'], 'status': 'created'}
{'created': 1509816949.0311673, 'generating_options': ['--input_folder=/data/CaloGAN/weigths/2', '--output_folder=/data/CaloGAN/outputs/2', '--sets=10', '--showers=100000'], 'training_options': ['--output_folder=/data/CaloGAN/weigths/2', '--nb-epochs=50', '--disc-lr=0.0001', '--gen-lr=0.00001', '', 'pion.yaml'], 'status': 'crea

In [17]:
def get_training_job():
    my_query={
        "size": 1,
        "query":{ "term": {"status":"created"}   }
    }

    res = es.search(index=index_name, body=my_query )
    print(res)
    for r in res['hits']['hits']:
        print(r)

In [ ]:
def get_generating_job():
    pass

In [ ]:
def get_transfering_job():
    pass

In [ ]:
def set_done(id):
    pass